In [8]:
torchvision.datasets.CIFAR10(root='../data', train=True, download=True)
torchvision.datasets.CIFAR10(root='../data', train=False, download=True)

100.0%


Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified


Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ../data
    Split: Test

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# This is a bit of magic gto make matplotlib figures appear inline
# in the notebook rather than in a new window
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [2]:
import cv2 as cv
import torchvision
import _pickle as pickle
import os

In [3]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import LinearSVC

In [6]:
data_dir = '../data/cifar-10-batches-py'

In [12]:
xs, ys = [], []
for b in range(1, 6):
    filename = os.path.join(data_dir, "data_batch_%d" % (b,))
    with open(filename, "rb") as f:
        datadict = pickle.load(f, encoding="latin1")
        X = datadict["data"]
        Y = datadict["labels"]
        X = X.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float")
        Y = np.array(Y)
    xs.append(X)
    ys.append(Y)

X_train = np.concatenate(xs)
y_train = np.concatenate(ys)

In [50]:
filename = os.path.join(data_dir, "test_batch")
with open(filename, 'rb') as f:
    datadict = pickle.load(f, encoding="latin1")
    X = datadict['data']
    print(X.size)
    Y = datadict['labels']
    X_test = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
    y_test = np.array(Y)


30720000
30720000


In [17]:
classifier = LinearSVC(random_state = 42)
sift = cv.SIFT_create()
kmeans = MiniBatchKMeans(n_clusters=1000, random_state=42)

In [36]:
def sift_svm_train(x, y, sift, kmeans, classifier):
    des_list = []
    for img in x:
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        kps, des = sift.detectAndCompute(img, None)
        des_list.append(des)
    des = np.vstack([descriptors for descriptors in des_list if descriptors is not None])
    kmeans.fit(des)

    features = np.array([des_to_features(des, sift, kmeans, classifier) for des in des_list])
    classifier.fit(features, y)
    return classifier


def des_to_features(des, sift, kmeans, classifier):
    if des is not None:
        labels = kmeans.predict(des)
        features = np.bincount(labels, minlength=kmeans.n_clusters).astype(float)
        features /= features.sum()  # L1 normalization
    else:
        features = np.zeros(kmeans.n_clusters)
    return features

def predict(x, sift, kmeans, classifier):
    des_list = []
    for img in x:
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        kps, des = sift.detectAndCompute(img, None)
        des_list.append(des)
    features = np.array([des_to_features(des, sift, kmeans, classifier) for des in des_list])
    return classifier.predict(features)


In [47]:
print(X_train.size / 32 / 32 / 3)
print(y_train.size)

50000.0
50000


In [29]:
clf = sift_svm_train(X_train, y_train, sift, kmeans, classifier)

In [38]:
y_pred = predict(X_test, sift, kmeans, clf)

In [42]:
print(y_pred.size)
print(y_test.size)

10000
10000


In [39]:
print('accuracy :', (y_test == y_pred).sum() / y_test.size)

accuracy : 0.2943
